In [2]:
import os
import glob
import pandas as pd

# Step 1: Define the extracted data folder and the sub-band mapping
extracted_data_folder = 'Extracted_Data_New'
subbands = {'A5': 'A5', 'D3': 'D3', 'D4': 'D4', 'D5': 'D5'}

# Step 2: Initialize a list to store data rows for the CSV
data_rows = []

# Step 3: Process each folder in the Extracted Data folder
folders = glob.glob(os.path.join(extracted_data_folder, '*'))

for folder in folders:
    signal_name = os.path.basename(folder)  # Extract signal name (e.g., Z0010)
    row = {'Data': signal_name}  # Initialize a row dictionary

    # Step 4: Loop through the sub-bands to extract metrics
    for subband, renamed_subband in subbands.items():
        metrics_file = os.path.join(folder, f'{signal_name}{renamed_subband}.txt')

        if os.path.exists(metrics_file):
            # Read metrics from the file
            with open(metrics_file, 'r') as f:
                metrics = f.readlines()

            # Parse and store each metric in the row dictionary
            row[f'MAV_{renamed_subband}'] = float(metrics[1].split(':')[1].strip())
            row[f'Avg.Power_{renamed_subband}'] = float(metrics[2].split(':')[1].strip())
            row[f'SD_{renamed_subband}'] = float(metrics[3].split(':')[1].strip())
            row[f'MAV_Ratio_{renamed_subband}'] = float(metrics[4].split(':')[1].strip())
        else:
            print(f"Metrics file for {renamed_subband} not found in {folder}.")
            # Fill with NaN if the file doesn't exist
            row[f'MAV_{renamed_subband}'] = float('nan')
            row[f'Avg.Power_{renamed_subband}'] = float('nan')
            row[f'SD_{renamed_subband}'] = float('nan')
            row[f'MAV_Ratio_{renamed_subband}'] = float('nan')

    # Step 5: Add label (1 if A_extended, 0 otherwise)
    row['label'] = int(metrics[0].split(':')[1].strip())

    # Step 6: Append the row to the data_rows list
    data_rows.append(row)

# Step 7: Convert the data into a DataFrame and save it as a CSV file
df = pd.DataFrame(data_rows)

# Suffling the data
df = df.sample(frac=1).reset_index(drop=True)

# Step 8: Define the CSV file path and save the DataFrame
csv_file_path = 'wavelet_metrics_1.csv'
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved successfully at {csv_file_path}")


CSV file saved successfully at wavelet_metrics_1.csv
